### Script to generate FATES parameters using LHS

In [1]:
from scipy.stats import qmc
import numpy as np

import csv
import pandas as pd
import os
import netCDF4 as nc4
import sys
import shutil
from tempfile import TemporaryFile                                                                                                                                 
import argparse                                                                                                                                                                                                                                                                                                       
import tempfile 
import random
import re

import modp as mp


In [2]:
random.seed(32)

#### Read in min and max values for each parameter and pft 

In [3]:
param_ranges_full = pd.read_csv('/global/homes/j/jneedham/BCI_leafmr/param_dir/Ensemble_params_0324/BCI_2pfts_param_ranges_scalers_0324.csv')
param_ranges = param_ranges_full[['param', 'value_min', 'value_max', 'pft', 'organ']]

# number of parameters
n_params = len(param_ranges)

# number of PFTs - some are global so subtract one
n_pfts = len(pd.unique(param_ranges['pft'])) - 1

param_names = list(param_ranges.param)
pfts = list(param_ranges.pft)
organs = list(param_ranges.organ)

print(param_ranges)

                              param  value_min  value_max  pft  organ
0  fates_maintresp_nonleaf_baserate   0.000002   0.000003    0    NaN
1      fates_mort_understorey_death   0.000000   1.000000    0    NaN
2                  fates_mort_bmort   0.020000   0.060000    1    NaN
3           fates_mort_bmort_scaler   0.200000   0.900000    2    NaN
4                      fates_grperc   0.110000   0.350000    1    NaN
5                fates_wood_density   0.300000   0.500000    1    NaN
6               fates_turnover_leaf   1.200000   2.000000    1    NaN
7             fates_turnover_branch  30.000000  80.000000    1    NaN
8             fates_leaf_vcmax25top  40.000000  72.000000    2    NaN


In [4]:
n_inst = 200

sampler = qmc.LatinHypercube(d=n_params)
sample = sampler.random(n=n_inst)

# scale to parameter ranges
l_bounds = param_ranges['value_min']
u_bounds = param_ranges['value_max']

scaled_sample = qmc.scale(sample, l_bounds, u_bounds)

print(scaled_sample.shape)

(200, 9)


In [5]:
## Read in defaut FATES file - note that this is the default for FATES but with:
# - updated allometries for tropical PFTs
# - size bins that are consistent with the DBEN protocol. 
# - supplemental seed rain
# - updated vai bins
# - atkin respiration 
# - size dependent mortality 

input_fname = '/global/homes/j/jneedham/BCI_leafmr/param_dir/Ensemble_params_0324/fates_params_bci_2pft.nc'

# for each sample 
for i in range(0,n_inst) :
    
    # final parameter file name
    fout = '/global/homes/j/jneedham/BCI_leafmr/param_dir/Ensemble_params_0324/fates_params_bci_ens_{0}.nc'.format(i)
    
    shutil.copyfile(input_fname, fout)                                                                                                                             
   
    # loop through each parameter and apply either to the correct pft or globally
    for j in range(0, n_params) : 
        
        var = param_names[j]
        pft = pfts[j]
        organ = organs[j]
        
        val = scaled_sample[i, j]
        
        # Bmort is sampled for PFT 1. A scaler is sampled for PFT 2 which is 
        # multiplied by the sampled value for PFT  1. 
        if var == 'fates_mort_bmort_scaler' :
            val = val * scaled_sample[i, (j-1)]
            #print('ST: ', val, '  LD: ',  scaled_sample[i, (j-1)])
            pft = 2
            var  = 'fates_mort_bmort'         
            
        mp.main(var = var, pft = pft, fin = fout, val = val, 
                    fout = fout, O = 1, organ = organ)
            
        if var == 'fates_wood_density' and pft == 1 : 
            pft = pft + 1
            val = val * 1.5
            mp.main(var = var, pft = pft, fin = fout, val = val, 
                    fout = fout, O = 1, organ = organ)
               
        if var == 'fates_turnover_leaf' and pft == 1 : 
            pft = pft + 1
            val = val * 1.5
            mp.main(var = var, pft = pft, fin = fout, val = val, 
                    fout = fout, O = 1, organ = organ)
            
        if var == 'fates_turnover_branch' and pft == 1 : 
            pft = pft + 1
            val = val * 1.5
            mp.main(var = var, pft = pft, fin = fout, val = val, 
                    fout = fout, O = 1, organ = organ)
            
      

In [6]:
print(scaled_sample.shape)

(200, 9)


In [7]:
tmp = scaled_sample[0:50,:]

df  = pd.DataFrame(data=tmp, columns = param_names)

